In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

In [2]:
rating = pd.read_csv('data/rating_complete.csv')

In [3]:
anime_info = pd.read_csv('data/anime.csv', usecols=['MAL_ID', 'Name'])

In [6]:
anime_info = anime_info.rename(columns={"MAL_ID": "anime_id"})

In [7]:
rating = rating.merge(anime_info, left_on = 'anime_id', right_on = 'anime_id', how = 'left')

In [8]:
rating

,user_id,anime_id,rating,Name
0,0,430,9,Fullmetal Alchemist: The Conqueror of Shamballa
1,0,1004,5,Kanojo to Kanojo no Neko
2,0,3010,7,Kaiketsu Zorro
3,0,570,7,Jin-Rou
4,0,2762,9,Igano Kabamaru
...,...,...,...,...
57633273,353404,502,8,Dragon Ball Movie 1: Shen Long no Densetsu
57633274,353404,987,4,Dragon Ball GT: Gokuu Gaiden! Yuuki no Akashi ...
57633275,353404,225,8,Dragon Ball GT
57633276,353404,243,7,Gravitation


In [9]:
count = rating['user_id'].value_counts()
count1 = rating['anime_id'].value_counts()
rating = rating[rating['user_id'].isin(count[count >= 500].index)].copy()
rating = rating[rating['anime_id'].isin(count1[count1 >= 200].index)].copy()

In [11]:
rating.shape

(18754914, 4)

In [12]:
piviot_table = rating.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)
piviot_table

user_id,17,19,42,111,112,146,156,172,198,222,...,353300,353304,353311,353318,353325,353326,353328,353365,353395,353398
Name,,,,,,,,,,,,,,,,,,,,,
"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Eiji""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,8.0,0.0
xxxHOLiC Rou,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
xxxHOLiC Shunmuki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0


In [14]:
piviot_table_matrix = csr_matrix(piviot_table.values)

In [15]:
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [16]:
random_anime = np.random.choice(piviot_table.shape[0]) 

In [17]:
random_anime

83

In [22]:
for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(piviot_table.index[random_anime]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, piviot_table.index[suggestions.flatten()[i]], distance.flatten()[i]))

Recommendations for 91 Days:

1: Bungou Stray Dogs, with distance of 0.33797393567752343:
2: Boku dake ga Inai Machi, with distance of 0.34497385333524533:
3: Death Parade, with distance of 0.34555512589156756:
4: Mob Psycho 100, with distance of 0.3543247157852042:
5: Zankyou no Terror, with distance of 0.35547261098858185:


In [35]:
test = list(piviot_table.index)

In [36]:
test.index('91 Days')

83

In [39]:
def predict(anime):
    indexes = list(piviot_table.index)
    anime_index = indexes.index(anime)
    query = piviot_table.iloc[anime_index, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=6)
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(piviot_table.index[random_anime]))
        else:
            print('{0}: {1}:'.format(i, piviot_table.index[suggestions.flatten()[i]]))

In [40]:
predict('Bungou Stray Dogs')

Recommendations for 91 Days:

1: Bungou Stray Dogs 2nd Season, with distance of 0.09658215041233831:
2: Noragami Aragoto, with distance of 0.2683208380420232:
3: Noragami, with distance of 0.2714046693369734:
4: Boku no Hero Academia, with distance of 0.2717715465766015:
5: Owari no Seraph, with distance of 0.28954979329292707:


In [52]:
knnPickle = open('knnpickle_file', 'wb') 
pickle.dump(model, knnPickle) 
knnPickle.close()